# scrapy_电影天堂多页数据下载

In [ ]:
scrapy startproject scrapy_movie_049

cd scrapy_movie_049/scrapy_movie_049/spiders

scrapy genspider mv https://www.dytt8.net/html/gndy/china/index.html

scrapy genspider mv https://btwuji.com/html/gndy/china/index.html


In [ ]:
# scrapy_movie_049/scrapy_movie_049/spiders/mv.py

# 要第一页电影的名字和电影链接后一页中电影海报的图片

import scrapy
from scrapy_movie_049.items import ScrapyMovie049Item

class MvSpider(scrapy.Spider):
    name = "mv"
    allowed_domains = ["btwuji.com"]
    start_urls = ["https://btwuji.com/html/gndy/china/index.html"]

    def parse(self, response):
        # 要第一个的名字和第二页的图片

        # 获取第一页各个电影的信息
        a_list = response.xpath('//div[@class="co_content8"]//td[2]//a[2]')
        for a in a_list:
            # 获取第一页的name 和 要点击的链接
            name = a.xpath('./text()').extract_first()
            src = a.xpath('./@href').extract_first()
            # print(name, src)

            # 第二页的地址（即电影的链接地址）
            url = 'https://btwuji.com' + src
            # print(url)

            # 对第二页的链接发起访问 通过meta参数传递name 传递给parse_second
            yield scrapy.Request(url, callback=self.parse_second, meta={'name': name})


    def parse_second(self, response):
        # 获取第二页的图片

        # 有坑，这里span失败不了，返回的都是None
        # 注意：如果拿不到数据的情况下，一定要检查你的xpath语法是否正确
        # src = response.xpath('//div[@id="Zoom"]/span/img/@src').extract_first()
        src = response.xpath('//div[@id="Zoom"]//img/@src').extract_first()
        print(src)

        # 接受到请求的那个meta参数的值
        name = response.meta['name']

        movie = ScrapyMovie049Item(src=src, name=name)

        yield movie


In [ ]:
# scrapy_movie_049/scrapy_movie_049/items.py

# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class ScrapyMovie049Item(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    name = scrapy.Field()
    src = scrapy.Field()



In [ ]:
# scrapy_movie_049/scrapy_movie_049/pipelines.py
# 注意要在settings.py中开启这个管道

# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://docs.scrapy.org/en/latest/topics/item-pipeline.html


# useful for handling different item types with a single interface
from itemadapter import ItemAdapter


class ScrapyMovie049Pipeline:

    def open_spider(self, spider):
        self.fp = open('movie.json', 'w', encoding='utf-8')

    def process_item(self, item, spider):
        self.fp.write(str(item))
        
        return item

    def close_spider(self, spider):
        self.fp.close()